# Download etopo22 topography

This Jupyter notebook is available in `$GTT/topo/fetch_etopo22.ipynb` within the
[GeoClaw Tsunami Tutorial](https://rjleveque.github.io/geoclaw_tsunami_tutorial).

It creates the file `$GTT/topo/topofiles/etopo22_30s_-130_-122_40_50_30sec.asc`, which can be used as a [topofile](https://www.clawpack.org/topo.html) in GeoClaw. The resolution is 30 arcsecond.

The etopo22 dataset is described on the webpage
[ETOPO 2022 15 Arc-Second Global Relief Model](https://data.noaa.gov/metaview/page?xml=NOAA/NESDIS/NGDC/MGG/DEM//iso/xml/etopo_2022.xml&view=getDataView&header=none).

Note that there is also a 15 arcsecond database, but it is split up into tiles and so it's a bit harder to download an arbitrary subset by the approach used here, and you may have to download pieces and stitch them together.

For most purposes 30" topography is more than adequate in the ocean, since in GeoClaw the topofile used for the deep ocean is usually augmented by one or more coastal DEMs at much higher resolution.

Note: This topofile can alternatively be downloaded using the `$GTT/topo/fetch_topo1.py` script.

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
import os
from clawpack.geoclaw import topotools

Set the limits of the domain and resolution desired:

In [ ]:
extent = [-130, -122, 40, 50]
coarsen = 1
arcsec = coarsen * 30
print('Will download etopo22_30s data at %i arcsecond resolution' % arcsec)

In [ ]:
url_thredds = 'https://www.ngdc.noaa.gov/thredds/dodsC/global/ETOPO2022/30s/30s_bed_elev_netcdf/ETOPO_2022_v1_30s_N90W180_bed.nc'
topo = topotools.read_netcdf(url_thredds, extent=extent,
                             coarsen=coarsen, verbose=True)

In [ ]:
name = 'etopo22_30s_%s_%s_%s_%s_%ssec' % tuple(extent + [arcsec])
fname = 'topofiles/%s.asc' % name
topo.write(fname, topo_type=3, header_style='asc',
           grid_registration='llcorner', Z_format='%.0f')
print('Created ',fname)

## Make a plot

In [ ]:
figure(figsize=(10,8))
ax = axes()
topo.plot(axes=ax, limits=(-3000,1000),
          cb_kwargs={'extend':'both','shrink':0.7})
title(name)
fname = fname.replace('.asc', '.png')
savefig(fname)
print('Created ', fname)

## Make a file containing the shoreline

This is only done so that the shoreline can be read in and plotted in other notebooks, e.g. [](../dtopo/ASCE_SIFT_Region2).  This saves the shoreline as a collection of piecewise linear segments, with x and y values in `csz_shore[:,0]` and `csz_shore[:,1]` respectively.

In [ ]:
csz_shore = topo.make_shoreline_xy()
fname = 'topofiles/csz_shore.txt'
savetxt(fname, csz_shore)
print('saved csz_shore from etopo1 in ',fname)